# **Tamil Nadu Plantation Analysis 2015-16**

This notebook analyzes plantation data for Tea, Bamboo, and Rubber in Tamil Nadu districts for the year 2015-16.

## Problem Statement
1. Analyze districts with maximum and minimum production of:
   - Tea
   - Bamboo
   - Rubber
2. Calculate total area of each plantation under all districts
3. Identify relationships between bamboo, tea and rubber plantations in each district

In [2]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import regexp_replace, substring, lit, col, when, expr, split
import re

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 4, Finished, Available, Finished)

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("TN Plantation Analysis") \
    .getOrCreate()

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 5, Finished, Available, Finished)

## **Data Loading and Cleaning Functions**

In [4]:
def clean_bamboo_data(df):
    """Clean Bamboo plantation data by removing special characters and alphabets"""
    # Clean District column - remove special characters
    df = df.withColumn('District', 
                       regexp_replace('District', '[^a-zA-Z\s]', ''))
    
    # Clean Area column - remove alphabets
    df = df.withColumn('Area', 
                       regexp_replace('Area', '[^0-9]', ''))
    
    # Convert Area to integer
    df = df.withColumn('Area', col('Area').cast(IntegerType()))
    
    return df


def format_sequence_file(data):
    # Replace every 5th delimiter with a newline character
    formatted_data = re.sub(r'((?:[^|]*\|){4}[^|]*)\|', r'\1\n', data)
    return formatted_data

def process_tea_data(df):
    """Process tea plantation data by handling sequence file format"""
    # Convert the string data into rows using split
    # Note: When reading text files, the column name is 'value'
    df = df.select(split(col("value"), "\\|").alias("columns"))
    
    # Extract columns from the array and handle 'NA' values
    df = df.select(
        col("columns")[0].cast(IntegerType()).alias("Serial_Number"),
        col("columns")[1].alias("District"),
        when(col("columns")[2] == "NA", None)
        .otherwise(col("columns")[2].cast(IntegerType())).alias("Area"),
        when(col("columns")[3] == "NA", None)
        .otherwise(col("columns")[3].cast(IntegerType())).alias("Production")
    )
    
    # Filter out rows where both Area and Production are NULL
    # df = df.filter((col("Area").isNotNull()) & (col("Production").isNotNull()))
    
    return df







StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 6, Finished, Available, Finished)

In [5]:
# Define schema for all datasets
schema = StructType([
    StructField("Serial_Number", IntegerType(), True),
    StructField("District", StringType(), True),
    StructField("Area", IntegerType(), True),
    StructField("Production", IntegerType(), True)
])

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 7, Finished, Available, Finished)

In [6]:
# Load and process Bamboo plantation data
bamboo_df = spark.read.csv('Files/TN_Plantation/bamboo-plantation-15-16.txt', 
                          schema=schema, 
                          sep='|')
bamboo_df = clean_bamboo_data(bamboo_df)
bamboo_df = bamboo_df.withColumn('Plantation_Type', lit('Bamboo'))
bamboo_df = bamboo_df.withColumn('Productivity', 
                                 col('Production') / col('Area'))

# Load and process Tea plantation data
tea_df = spark.read.text('Files/TN_Plantation/tea-plantation-15-16.txt').collect()
tea_df_str = ''.join([row.value for row in tea_df])
formatted_df = format_sequence_file(tea_df_str)
# print(formatted_df)

# Convert the formatted string into a list of rows
formatted_data_list = formatted_df.split('\n')

# Create a DataFrame from the list of rows
formatted_df = spark.createDataFrame(formatted_data_list, StringType()).toDF("value")
# print(formatted_df)

tea_df = process_tea_data(formatted_df)
tea_df = tea_df.withColumn('Plantation_Type', lit('Tea'))
tea_df = tea_df.withColumn('Productivity', 
                          when(col('Area') > 0, col('Production') / col('Area'))
                          .otherwise(None))
display(tea_df)

# Load and process Rubber plantation data
rubber_df = spark.read.csv('Files/TN_Plantation/rubber-plantation-15-16.txt',
                          schema=schema,
                          sep='|',
                          header=True)
rubber_df = rubber_df.withColumn('Plantation_Type', lit('Rubber'))
rubber_df = rubber_df.withColumn('Productivity', 
                                col('Production') / col('Area'))
                                
display(rubber_df)
# Union all dataframes
combined_df = bamboo_df.union(tea_df).union(rubber_df)
combined_df.createOrReplaceTempView("plantations")

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f656573a-c6ca-40d6-b257-e1b6182c4853)

SynapseWidget(Synapse.DataFrame, e62a2b66-de9b-4670-9436-f3bc317d686c)

## **Analysis 1: Maximum and Minimum Production by Plantation Type**

In [7]:
# SQL query to find districts with max and min production for each plantation type
production_analysis = spark.sql("""
    WITH RankedProductions AS (
        SELECT 
            Plantation_Type,
            District,
            Production,
            RANK() OVER (PARTITION BY Plantation_Type ORDER BY Production DESC) as max_rank,
            RANK() OVER (PARTITION BY Plantation_Type ORDER BY Production ASC) as min_rank
        FROM plantations
    )
    SELECT 
        Plantation_Type,
        MAX(CASE WHEN max_rank = 1 THEN District END) as Max_Production_District,
        MAX(CASE WHEN max_rank = 1 THEN Production END) as Max_Production,
        MAX(CASE WHEN min_rank = 1 THEN District END) as Min_Production_District,
        MAX(CASE WHEN min_rank = 1 THEN Production END) as Min_Production
    FROM RankedProductions
    GROUP BY Plantation_Type
    ORDER BY Plantation_Type
""")

production_analysis.show()

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 9, Finished, Available, Finished)

+---------------+-----------------------+--------------+-----------------------+--------------+
|Plantation_Type|Max_Production_District|Max_Production|Min_Production_District|Min_Production|
+---------------+-----------------------+--------------+-----------------------+--------------+
|         Bamboo|           Nagapattinam|          6854|            TheNilgiris|          NULL|
|         Rubber|            Kanyakumari|         30027|           Virudhunagar|          NULL|
|            Tea|            Kanyakumari|         30027|           Virudhunagar|          NULL|
+---------------+-----------------------+--------------+-----------------------+--------------+



## **Analysis 2: Total Area by Plantation Type**

In [8]:
# Calculate total area for each plantation type
area_analysis = spark.sql("""
    SELECT 
        Plantation_Type,
        SUM(Area) as Total_Area
    FROM plantations
    GROUP BY Plantation_Type
    ORDER BY Total_Area DESC
""")

area_analysis.show()

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 10, Finished, Available, Finished)

+---------------+----------+
|Plantation_Type|Total_Area|
+---------------+----------+
|            Tea|     27016|
|         Rubber|     27016|
|         Bamboo|       606|
+---------------+----------+



## **Analysis 3: Relationship Analysis between Plantations**

In [9]:
# Pivot the data to analyze relationships
relationship_analysis = spark.sql("""
    WITH PivotedData AS (
        SELECT 
            District,
            MAX(CASE WHEN Plantation_Type = 'Bamboo' THEN Area END) as Bamboo_Area,
            MAX(CASE WHEN Plantation_Type = 'Tea' THEN Area END) as Tea_Area,
            MAX(CASE WHEN Plantation_Type = 'Rubber' THEN Area END) as Rubber_Area
        FROM plantations
        GROUP BY District
    )
    SELECT
        District,
        Bamboo_Area,
        Tea_Area,
        Rubber_Area,
        CASE 
            WHEN Bamboo_Area > Tea_Area AND Bamboo_Area > Rubber_Area THEN 'Bamboo Dominant'
            WHEN Tea_Area > Bamboo_Area AND Tea_Area > Rubber_Area THEN 'Tea Dominant'
            WHEN Rubber_Area > Bamboo_Area AND Rubber_Area > Tea_Area THEN 'Rubber Dominant'
            ELSE 'Mixed'
        END as Dominant_Plantation
    FROM PivotedData
    ORDER BY District
""")

display(relationship_analysis)

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4db4ccf3-03bc-4a9e-9aed-10d32d4704e6)

## **Correlation Analysis using Spark ML**

In [16]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
import numpy as np

# Prepare data for correlation analysis
pivot_df = relationship_analysis.select('Bamboo_Area', 'Tea_Area', 'Rubber_Area')

# Remove rows with null values
pivot_df = pivot_df.dropna()

# Create vector assembler
assembler = VectorAssembler(
    inputCols=['Bamboo_Area', 'Tea_Area', 'Rubber_Area'],
    outputCol='features')

# Create vector column
vector_df = assembler.transform(pivot_df)

# Calculate correlation matrix
matrix = Correlation.corr(vector_df, 'features').head()[0]

# Convert matrix to numpy array
matrix_np = np.array(matrix.toArray())

# Create a DataFrame from the numpy array
columns = ['Bamboo_Area', 'Tea_Area', 'Rubber_Area']
correlation_df = spark.createDataFrame(matrix_np.tolist(), columns)

# Display the DataFrame
correlation_df.show(truncate=False)
display(correlation_df)

StatementMeta(, b47a42c9-225c-4df4-a787-37e6df1c259d, 18, Finished, Available, Finished)

+-----------+------------------+------------------+
|Bamboo_Area|Tea_Area          |Rubber_Area       |
+-----------+------------------+------------------+
|1.0        |-1.0              |-1.0              |
|-1.0       |1.0               |1.0000000000000002|
|-1.0       |1.0000000000000002|1.0               |
+-----------+------------------+------------------+



SynapseWidget(Synapse.DataFrame, 9af89e18-4e25-4e87-9aa0-8fac611154a1)